In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

from alibi.explainers import AnchorTabular

from sklearn.model_selection import train_test_split

W0724 08:27:22.263968 4602295744 deprecation_wrapper.py:119] From /Users/kennedymurphy/venv/lib/python3.7/site-packages/alibi/explainers/cem.py:19: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
train = pd.read_csv("../data/train_features.csv")
train.set_index('PassengerId', inplace=True)

test = pd.read_csv("../data/test_features.csv")
test.set_index("PassengerId", inplace=True)

X = train[[c for c in train.columns if c != 'Survived']].values
y = train.Survived.values

feature_names = train[[c for c in train.columns if c != 'Survived']].columns

test = test[feature_names]

In [3]:
model = xgb.XGBClassifier()
model = model.fit(X, y)

In [4]:
from alibi.explainers import AnchorTabular

explainer = AnchorTabular(
    predict_fn=lambda x: model.predict(x),
    feature_names=feature_names)

explainer.fit(X, disc_perc=[25, 50, 75])

In [5]:
# Generate explanations
records = []
num_records = test.shape[0]
idx = test.index
values = test.values

for i in range(num_records):
    explanation = explainer.explain(
        values[i], threshold=0.95, max_anchor_size=5)
    
    explanation_summary = {
        'prediction': model.predict(values[i,].reshape(1,-1))[0],
        'anchor': " AND ".join([str(x) for x in explanation['names']]),
        'precision': float(round(explanation['precision'], 4)),
        'coverage': float(round(explanation['coverage'], 4))
    }
    
    records.append(explanation_summary)

W0724 08:27:26.829955 4602295744 anchor_base.py:618] Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible anchor.
W0724 08:27:28.009369 4602295744 anchor_base.py:618] Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible anchor.
W0724 08:27:35.970633 4602295744 anchor_base.py:618] Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible anchor.
W0724 08:27:39.744749 4602295744 anchor_base.py:618] Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible anchor.
W0724 08:27:46.453356 4602295744 anchor_base.py:618] Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible anchor.
W0724 08:27:55.429753 4602295744 anchor_base.py:618] Could not find an anchor satisfying the 0.95 precision constraint. Now returning the best non-eligible anchor.
W0724 08:28:16.3

In [6]:
records = pd.DataFrame(records).sort_values('coverage')

In [14]:
records.loc[365].to_dict()

{'anchor': 'Fare <= 8.05 AND Miss <= 0.00 AND Embarked_C <= 0.00 AND Master <= 0.00',
 'coverage': 0.3898,
 'precision': 0.8767,
 'prediction': 0.0}

In [8]:
records.loc[244].to_dict()

{'anchor': 'Master > 0.00 AND Infant <= 0.00 AND Child <= 0.00',
 'coverage': 0.0086,
 'precision': 0.9553,
 'prediction': 1.0}

In [23]:
records[records['anchor'] == 'Fare <= 8.05 AND Miss <= 0.00 AND Embarked_C <= 0.00 AND Master <= 0.00']

,anchor,coverage,precision,prediction
365,Fare <= 8.05 AND Miss <= 0.00 AND Embarked_C <...,0.3898,0.8767,0.0


In [23]:
" AND ".join(['Female <= 0.00', 'Fare <= 8.05'])

'Female <= 0.00 AND Fare <= 8.05'